# Imports and environment

In [10]:
import os
import sys
import re
from pathlib import Path

from dotenv import load_dotenv
from openai import OpenAI
from markdown_pdf import MarkdownPdf, Section

# Load environment variables (e.g. OPENAI_API_KEY)
load_dotenv()

True

# User‐configurable variables (formerly CLI args)

In [ ]:
output_file   = "../results/Communication skills as a snowflake data consultant.pdf"           # --output
model         = "gpt-4.1-mini"         # --model
toc_level     = 3                      # --toc-level
optimize      = False                  # --optimize
use_web_search= True                   # --no-web-search → False
max_output_tokens = 2000
prompts_filename = "../prompts.txt"


# Helper function definitions

In [12]:
def generate_markdown_from_prompt(user_prompt, client):
    try:
        prompt = [
            {"role": "system", "content": (
                # "You are a coach helping me to improve my soft skills as Snowflake Data Architects and Solutions Architects in a consultancy context. "
                "Provide output in Markdown. "
                "Use **bold** text for headers and not don't use '#' or '#'s headers. "
                "Don't add line separators in response. "
                # "Add code snippets where needed."
                ""
            )},
            {"role": "user", "content": user_prompt},
        ]
        tools = [{"type": "web_search_preview"}] if use_web_search else []
        response = client.responses.create(
            model=model,
            tools=tools,
            input=prompt,
            max_output_tokens = max_output_tokens
        )
        text = f"Prompt: {user_prompt}\n\n" + response.output_text.strip()
        return text
    except Exception as e:
        print(f"AI request failed for node '{user_prompt}': {e}")
        return f"{user_prompt}\n\n---\n\n"

def process_prompts(prompts_text, client, pdf, css):
    lines = prompts_text.strip().splitlines()
    
    len_lines = len(lines)
    
    section_md = ""
    for idx, line in enumerate(lines, start=1):
        stripped = line.lstrip()
        # skip empty lines
        if not stripped:
            continue
        # skip lines starting with '#'
        elif stripped.startswith("#"):
            section_md = f"{section_md}\n\n{stripped}"
        else:
            print(f"Processing Line {idx}/{len_lines} ({round(idx*100/len_lines)}%)...")
            md = generate_markdown_from_prompt(stripped, client)
            section_md = f"{section_md}\n\n{md}\n\n"
            pdf.add_section(Section(section_md), user_css=css)
            section_md = ""

# Prompts

In [13]:
# Open the file in read mode and assign its contents to chatgpt_prompts
with open(prompts_filename, "r", encoding="utf-8") as f:
    chatgpt_prompts = f.read()

if fetch_filename_from_header:
    header_text = None
    for line in chatgpt_prompts.splitlines():
        if line.lstrip().startswith("# "):
            header_text = line.lstrip("# ").strip()
            break

    if header_text:
        # simple slugify: lowercase, drop bad chars, spaces→underscore
        def slugify(s):
            s = s.lower()
            s = re.sub(r"[^\w\s-]", "", s)
            return re.sub(r"\s+", "_", s).strip("_")

        base_dir = os.path.dirname(output_file)
        output_file = os.path.join(base_dir, f"{slugify(header_text)}.pdf")
        print(f"📝  Using header for filename → {output_file}")
    else:
        print("⚠️  fetch_filename_from_header is True, but no '# ' header found; using default output_file")

# Optional: print to verify
# print(chatgpt_prompts)

📝  Using header for filename → ..\communication_skills_as_a_snowflake_data_consultant.pdf


# Instantiate client, load YAML, and generate Markdown sections

In [ ]:
from pathlib import Path

api_key = os.getenv("OPENAI_API_KEY")
css = Path("custom.css").read_text(encoding="utf-8")
pdf = MarkdownPdf(toc_level=toc_level, optimize=optimize)

if not api_key:
    print("Environment variable OPENAI_API_KEY is not set.")
    sys.exit(1)

client = OpenAI(api_key=api_key)
            
process_prompts(chatgpt_prompts, client, pdf, css)


# Save as PDF

In [9]:
try:
    pdf.save(output_file)
    print(f"PDF successfully saved to {output_file}")
except Exception as e:
    print(f"Failed to save PDF: {e}")
    raise

Failed to save PDF: name 'pdf' is not defined


NameError: name 'pdf' is not defined